<a href="https://colab.research.google.com/github/jarvisx17/EPO/blob/main/EPO_model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install transformers

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder

In [2]:
# Step 1: Preprocess the data
df = pd.read_csv('/content/Final_df2_EPO.csv')  # Replace 'your_dataset.csv' with your actual dataset file
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 512  # Maximum sequence length for input text

texts = df['Cited Text'].tolist()
claims = df['rel_Claim'].tolist()
labels = df['Label'].tolist()

In [3]:
#Label Encoding
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels

array([0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1])

In [4]:
input_ids = []
attention_masks = []

for text, claim in zip(texts, claims):
    encoded_dict = tokenizer.encode_plus(
        text,
        claim,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_masks, labels)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [5]:
# Step 2: Split the data
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Step 3: Load a pre-trained transformer model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Step 4: Create a classification model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
# Step 5: Train the model
batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}/{epochs} - Average Loss: {avg_loss}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20 - Average Loss: 0.5564049035310745
Epoch 2/20 - Average Loss: 0.47262194007635117
Epoch 3/20 - Average Loss: 0.35150160640478134
Epoch 4/20 - Average Loss: 0.25624828413128853
Epoch 5/20 - Average Loss: 0.2507883459329605
Epoch 6/20 - Average Loss: 0.20128446072340012
Epoch 7/20 - Average Loss: 0.143555399030447
Epoch 8/20 - Average Loss: 0.11200217436999083
Epoch 9/20 - Average Loss: 0.10982909798622131
Epoch 10/20 - Average Loss: 0.10579824354499578
Epoch 11/20 - Average Loss: 0.08278318960219622
Epoch 12/20 - Average Loss: 0.07817942695692182
Epoch 13/20 - Average Loss: 0.07560586743056774
Epoch 14/20 - Average Loss: 0.07380335219204426
Epoch 15/20 - Average Loss: 0.07836930966004729
Epoch 16/20 - Average Loss: 0.09409598866477609
Epoch 17/20 - Average Loss: 0.09803911484777927
Epoch 18/20 - Average Loss: 0.0479310667142272
Epoch 19/20 - Average Loss: 0.035560015588998795
Epoch 20/20 - Average Loss: 0.03104665270075202


In [7]:
# Step 6: Evaluate the model
model.eval()
total_predictions, correct_predictions = 0, 0

for batch in train_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_masks, labels = batch

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits

    predicted_labels = torch.argmax(logits, dim=1)
    total_predictions += labels.size(0)
    correct_predictions += (predicted_labels == labels).sum().item()

accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


In [8]:
# Step 6: Evaluate the model
model.eval()
total_predictions, correct_predictions = 0, 0

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, attention_masks, labels = batch

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits

    predicted_labels = torch.argmax(logits, dim=1)
    total_predictions += labels.size(0)
    correct_predictions += (predicted_labels == labels).sum().item()

accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy}')

Accuracy: 0.7857142857142857


In [9]:
# torch.save(model, 'model_weights.pth')
# Save the model weights
torch.save(model.state_dict(), 'model_weights.pth')

# Save the entire model (including architecture and optimizer state)
torch.save(model, 'model.pth')

In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load the trained model weights
model.load_state_dict(torch.load('model_weights.pth'))  # Replace 'model_weights.pth' with the path to your trained model weights

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Text and claim to be classified
text = df.iloc[0][0]
claim = df.iloc[0][1]
actual = df.iloc[0][2]

# Tokenize and encode the input
input_ids = tokenizer.encode(text, claim, add_special_tokens=True, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
input_ids = input_ids.to(device)

# Perform inference
with torch.no_grad():
    outputs = model(input_ids=input_ids)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

predicted_label = predicted_labels.item()
if predicted_label == 0:
    prediction = "Negative"
else:
    prediction = "Positive"

print(f"Prediction: {prediction}  \nActual : {actual}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Prediction: Negative  
Actual : Negative


# Document Wise Inference

In [16]:
# Input Data

citation_text = '''[0028]    The direct sound calculator 101, the early reflected sound calculator 102, and the late reverberant sound calculator 103 calculate propagation characteristics relating to a direct sound, early reflected sounds, and late reverberant sounds, respectively. More specifically, the direct sound calculator 101 applies a simulation result of propagation characteristics corresponding to a direct sound propagation path of a space to an input signal that is received from the input terminal 115, and allocates resulting signals to plural output lines to which the respective multipliers 106 are connected.[0029]    The early reflected sound calculator 102 applies simulation results of propagation characteristics corresponding to respective propagation paths of early reflected sounds to an input signal that is received from the input terminal 115 via the acoustic characteristics approximation filter 104, and allocates resulting signals to plural output lines to which the respective multipliers 107 are connected.[0030]    The late reverberant sound calculator 103 applies simulation results of propagation characteristics corresponding to respective propagation paths of late reverberant sounds to the input signal that is received from the input terminal 115, and allocates resulting signals to plural output lines to which the non-correlation IR convolution filter 105 is connected.[0044]    Returning to Fig. 3 , the multipliers 106 are disposed downstream of the respective output lines 242 of the direct sound calculator 101. The multipliers 106 multiply audio signals (indicating a direct sound) that are output from the output terminals of the direct sound calculator 101 by particular coefficients specified by the controller 10. The multipliers 107 are disposed downstream of the respective output lines 242 of the early reflected sound calculator 102. The multipliers 107 multiply audio signals (indicating early reflected sounds) that are output from the output terminals of the early reflected sound calculator 102 by particular coefficients specified by the controller 10. The multipliers 108 are disposed downstream of the non-correlation IR convolution filter 105. The multipliers 108 multiply audio signals (indicating late reverberant sounds) that have been subjected to the filter processing of the non-correlation IR convolution filter 105 by particular coefficients specified by the controller 10. The sets of multipliers 106, 107, and 108 have roles of adjusting sound volume balance of a direct sound, early reflected sounds, and late reverberant sounds, respectively, according to the sets of coefficients specified by the controller 10.[0045]    The adders 109 add output audio signals of the multipliers 106 on the output lines 242 to output audio signals of the multipliers 107 on the corresponding output lines 242. That is, the adders 109 have a role of adding direct sounds to early reflected sounds on an output-system-by-output-system basis. The adders 110 add output audio signals of the adders 109 on the output lines 242 to output audio signals of the multipliers 108 on the corresponding output lines 242. That is, the adders 110 have a role of adding direct sounds, early reflected sounds, and late reverberant sounds together on an output-system-by-output-system basis. The adders 110 output, to the output stage speakers 80, output signals obtained by adding reverberant sound signals to the input signal.'''

claims = [
    "1. A sound signal processing method comprising:\nreceiving a sound signal;\ngenerating an early reflection sound control signal that reproduces an early reflection sound and a reverberant sound control signal that reproduces a reverberant sound from the sound signal;\ncontrolling a volume of the sound signal and distributing the sound signal to generate a direct sound control signal that reproduces a direct sound; and\nmixing the direct sound control signal, the early reflection sound control signal, and the reverberant sound control signal to generate an output signal.",
    "2. The sound signal processing method according to claim 1, further comprising:\nobtaining an impulse response of an early reflection sound measured in advance in a space and an impulse response of a reverberant sound measured in advance in the space;\nconvolving the impulse response of the early reflection sound with the sound signal to generate the early reflection sound control signal; and\nconvolving the impulse response of the reverberant sound with the sound signal to generate the reverberant sound control signal.",
    "3. The sound signal processing method according to claim 2, wherein the sound signal to be convolved with the impulse response of the reverberant sound is received from an omnidirectional microphone (12A, 12B, 12C).",
    "4. The sound signal processing method according to any one of claims 1 to 3, wherein receiving the sound signal comprises receiving a line-inputted sound signal.",
    "5. The sound signal processing method according to any one of claims 1 to 3, wherein receiving the sound signal comprises receiving the sound signal from a microphone disposed on a performer.",
    "6. The sound signal processing method according to claim 5, wherein\nthe microphone is a directional microphone (11A, 11B, 11C).",
    "7. A sound signal processing device comprising:\na sound signal obtainer (21) that obtains a sound signal;\nan early reflection sound control signal generator (24A) that generates an early reflection sound control signal that reproduces an early reflection sound from the sound signal;\na reverberant sound control signal generator (24B) that generates a reverberant sound control signal that reproduces a reverberant sound from the sound signal;\na direct sound control signal generator (23D) that generates a direct sound control signal that reproduces a direct sound by controlling a volume of the sound signal and distributing the sound signal; and\nan output signal generator (26) that generates an output signal by mixing the direct sound control signal, the early reflection sound control signal, and the reverberant sound control signal.",
    "8. The sound signal processing device according to claim 7, further comprising:\nan impulse response obtainer (151) that obtains an impulse response of an early reflection sound measured in advance in a space and an impulse response of a reverberant sound measured in advance in the space,\nwherein\nthe early reflection sound control signal generator (24A) convolves the impulse response of the early reflection sound with the sound signal to generate the early reflection sound control signal; and\nthe reverberant sound control signal generator (24B) convolves the impulse response of the reverberant sound with the sound signal to generate the reverberant sound control signal.",
    "9. The sound signal processing device according to claim 8, wherein\nthe sound signal to be convolved with the impulse response of the reverberant sound is received from an omnidirectional microphone (12A, 12B, 12C).",
    "10. The sound signal processing device according to any one of claims 7 to 9, wherein\nthe sound signal obtainer obtains the sound signal by receiving a line-inputted sound signal.",
    "11. The sound signal processing device according to any one of claims 7 to 9, wherein\nthe sound signal obtainer obtains the sound signal by receiving the sound signal from a microphone disposed on a performer.",
    "12. The sound signal processing device according to claim 11, wherein\nthe microphone is a directional microphone (11A, 11B, 11C).",
    "13. A sound signal processing program for causing a sound signal processing device to execute processing of:\nreceiving a sound signal;\ngenerating an early reflection sound control signal that reproduces an early reflection sound and a reverberant sound control signal that reproduces a reverberant sound from the sound signal;\ncontrolling a volume of the sound signal and distributing the sound signal to generate a direct sound control signal that reproduces a direct sound; and\nmixing the direct sound control signal, the early reflection sound control signal, and the reverberant sound control signal to generate an output signal."
] 

# Citation details:  4 ['X', 'Y'] ['1,4,7,10,13', '2,3,5,6,8,9,11,12']

In [25]:
positive_claims = []
negative_claims = []
for i in claims:
  # Text and claim to be classified
  text = citation_text
  claim = i
  # Tokenize and encode the input
  input_ids = tokenizer.encode(text, claim, add_special_tokens=True, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
  input_ids = input_ids.to(device)

  # Perform inference
  with torch.no_grad():
      outputs = model(input_ids=input_ids)
      logits = outputs.logits
      predicted_labels = torch.argmax(logits, dim=1)

  predicted_label = predicted_labels.item()
  if predicted_label == 0:
      prediction = "Negative"
      negative_claims.append(i)
  else:
      prediction = "Positive"
      positive_claims.append(i)
print('\n\n')
print("Positive claims : ")
print('\n')
for j in positive_claims:
  print(f"claim :\n {j}")
print('\n\n')
print('Negative claims :')
print('\n')
for j in negative_claims:
  print(f"claim :\n {j}")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai




Positive claims : 


claim :
 1. A sound signal processing method comprising:
receiving a sound signal;
generating an early reflection sound control signal that reproduces an early reflection sound and a reverberant sound control signal that reproduces a reverberant sound from the sound signal;
controlling a volume of the sound signal and distributing the sound signal to generate a direct sound control signal that reproduces a direct sound; and
mixing the direct sound control signal, the early reflection sound control signal, and the reverberant sound control signal to generate an output signal.
claim :
 7. A sound signal processing device comprising:
a sound signal obtainer (21) that obtains a sound signal;
an early reflection sound control signal generator (24A) that generates an early reflection sound control signal that reproduces an early reflection sound from the sound signal;
a reverberant sound control signal generator (24B) that generates a reverberant sound control signal 